# Herunterladen von Twitter-Daten zum Thema Fahrrad und DB

In dieser Jupyter-Datei werden Daten aus Twitter heruntergeladen und in einer Pickle-Datei gespeichert. Sie werden in der Form "YYYY-MM-DD_tweets_DB_Fahrrad" im Ordner "Tweets" gespeichert. Die abgerufenen Daten in jeder Pickle-Datei sind nicht älter als 7 Tage zum Zeitpunkt des Abrufs bei Twitter. Zum Abrufen der Daten aus Twitter wird ein Developer-Account benötigt. In unserem Fall haben wir uns für den "V2 Access Levels - Essential" Account entschieden. Er bietet das herunterladen von 500.000 Tweets pro Montat und ist kostenlos nutzbar, gibt jedoch keine Tweets älter als 7 Tage aus.
Weitere Informationen zu den einzelnen Accounts gibt es hier: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api (Abgerufen: 18.07.2022)

Unsere Idee ist es daher, einzelne Pickle-Dateien für jede Woche (Beginn 01.05.2022) zu erstellen und aus ihnen ein Datenset zum Thema Fahrrad und Bahn zu erstellen, um diese Daten dann zu analysieren. Die Zusammenstellung der einzelnen Pickle-Dateien zu einem großen Datenset und die Analyse der Daten erfolgt dann in der Jupyter-Datei "Load_Datasets.ipynb".

## Vorgehensweise

Zunächst werden die benötigten Bibliotheken und Schlüssel für eine erfolgreiche Anbindung an Twitter importiert. Keys und Tokens können unter dem folgenden Link mithilfe eines gültigen Twitter-Accounts erstellt werden:  https://developer.twitter.com/en/portal/dashboard (Abgerufen: 18.07.2022)

In [1]:
from keys import *
import tweepy
import pandas as pd
import requests

Die Einstellungen des Clients, um eine erfolgreiche Anbindung an die Twitter-API zu ermöglichen, wurden nach folgendem Tutorial gesetzt: https://www.kirenz.com/post/2021-12-10-twitter-api-v2-tweepy-and-pandas-in-python/twitter-api-v2-tweepy-and-pandas-in-python/ (Abgerufen: 18.07.2022)

In [2]:

client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)


Um bestimmte Tweets von der Twitter-API anfordern zu können, muss eine Query übergeben werden, sodass die API dann gefilterte Tweets dementsprechend zurückgeben kann. Queries können unterschiedlich aufgebaut sein. Für weitere Informationen siehe: https://medium.com/@robguilarr/making-queries-to-twitter-api-on-tweepy-66afeb7184a4 (Abgerufen: 18.07.2022)

In der Funktion "get_tweets" wird eine Search-Query, sowie die maximale Anzahl der Rückgabe an gefundenen Tweets festgelegt. In dem Feld "tweet_fields" wird festgelegt, welche Informationen zu dem einzelnen, gefundenen Tweet geliefert werden sollen.

In [3]:
def get_tweets(query, max_results):
    tweets = client.search_recent_tweets(query=query, 
                                        tweet_fields=['attachments',
                                                        'author_id',
                                                        'context_annotations',
                                                        'conversation_id',
                                                        'created_at',
                                                        'geo',
                                                        'id',
                                                        'in_reply_to_user_id',
                                                        'lang',
                                                        'public_metrics',
                                                        'referenced_tweets',
                                                        'text'],
                                         max_results=max_results)
    return tweets


Mithilfe von Schlüsselwörtern zum Kontext Bahn und Fahrrad soll später eine Query erstellt werden. Aus diesem Grund wurde je eine Synonym-Liste für das Wort "Bahn" und "Fahrrad" erstellt.

In [4]:
keywords = {'bahn_context':['regio',
                            'Zug',
                            'sbahn',
                            'ubahn',
                            'Stadtbahn',
                            'Regionalbahn',
                            'Straßenbahn',
                            'Tram',
                            'Bus',
                            'Fernbus',
                            'Stadtbus',
                            'bahn',
                            'ÖPNV',
                            'ICE',
                            'IC',
                            'RE'],
            'bike_context':['Fahrrad',
                            'call a bike',
                            'rad',
                            'bike',
                            'ebike',
                            'Elektrorad',
                            'Elektrobike',
                            'Elektrofahrrad',
                            'pedelec',
                            'velo',
                            'Mountainbike',
                            'Trekkingbike',
                            'Radl']
           }

Nun wird mthilfe der Schlüsselwörter zum Kontext Bahn und Fahrrad die Search-Query erstellt und mit der Funktion "get_tweets" die Anfrage an die Twitter-API übergeben. Um einen groben Überblick über die gefilterten Tweets zu erhalten, wird die Anzahl der gefundenen Tweets in seinem jeweiligen Synonym-Paar aus Bahn und Fahrrad ausgegeben. Es bleibt dabei zu berücksichtigen, dass in jedem Paar nur maximal 100 Tweets stehen können, da die Anzahl durch den Essential-Account von Twitter beschränkt ist. 

In [5]:
# 192 Permutationen

max_results = 100 # maximum is 100 due to ESSENTIAL access 
tweets_data = pd.DataFrame()
for first_keyword in keywords['bahn_context']:
    for second_keyword in keywords['bike_context']:
        query = first_keyword + ' ' + second_keyword
        tweets = get_tweets(query,max_results)
        # Save data as dictionary
        tweets_dict = tweets.json()
        # check if tweets where found for this query
        if 'data' in tweets_dict:
            # Extract "data" value from dictionary
            new_tweet_data = tweets_dict['data']
            # Transform to pandas Dataframe
            df_new_tweet_data = pd.json_normalize(new_tweet_data)
            df_new_tweet_data['query'] = query
            # concat dataframes with previously loaded data
            tweets_data = pd.concat([tweets_data, df_new_tweet_data])
            print(len(df_new_tweet_data), 'results for query', query, 'returned')           
        else:
            print('0 results for query', query, 'returned')

print(len(tweets_data),'tweets where found with', tweets_data['id'].nunique(), 'unique tweet_ids')

6 results for query regio Fahrrad returned
0 results for query regio call a bike returned
2 results for query regio rad returned
3 results for query regio bike returned
0 results for query regio ebike returned
0 results for query regio Elektrorad returned
0 results for query regio Elektrobike returned
0 results for query regio Elektrofahrrad returned
1 results for query regio pedelec returned
2 results for query regio velo returned
0 results for query regio Mountainbike returned
0 results for query regio Trekkingbike returned
0 results for query regio Radl returned
99 results for query Zug Fahrrad returned
0 results for query Zug call a bike returned
74 results for query Zug rad returned
13 results for query Zug bike returned
3 results for query Zug ebike returned
0 results for query Zug Elektrorad returned
0 results for query Zug Elektrobike returned
0 results for query Zug Elektrofahrrad returned
2 results for query Zug pedelec returned
12 results for query Zug velo returned
1 result

1 results for query IC Fahrrad returned
0 results for query IC call a bike returned
12 results for query IC rad returned
20 results for query IC bike returned
0 results for query IC ebike returned
0 results for query IC Elektrorad returned
0 results for query IC Elektrobike returned
0 results for query IC Elektrofahrrad returned
0 results for query IC pedelec returned
7 results for query IC velo returned
0 results for query IC Mountainbike returned
0 results for query IC Trekkingbike returned
0 results for query IC Radl returned
7 results for query RE Fahrrad returned
99 results for query RE call a bike returned
100 results for query RE rad returned
99 results for query RE bike returned
99 results for query RE ebike returned
0 results for query RE Elektrorad returned
0 results for query RE Elektrobike returned
0 results for query RE Elektrofahrrad returned
1 results for query RE pedelec returned
99 results for query RE velo returned
5 results for query RE Mountainbike returned
0 result

Die erfolgreich heruntergeladenen Tweets werden nun in einer Pickle-Datei gespeichert. Sie werden nach dem Schema "YYYY-MM-DD_tweets_DB_Fahrrad" gespeichert unter dem Ordner "Tweets" gespeichert.

In [6]:
save_filename = pd.Timestamp.today().strftime('%Y-%m-%d') + '_tweets_DB_Fahrrad'
tweets_data.to_pickle('Tweets/' + save_filename)

Die Daten wurden bis dahin noch nicht nach ihrer Sprache gefiltert. Um hier einen groben Überblick zu bekommen, welche Sprache am meisten vorhanden ist, werden die Anzahl an Tweets nach ihrere Sprache ausgegeben.

In [7]:
tweets_data['lang'].value_counts()

de     953
en     882
fr     220
tr      15
qme      7
es       7
lv       6
und      5
it       4
tl       3
hi       3
ja       2
nl       2
ro       2
cs       2
zxx      2
sv       1
in       1
et       1
ko       1
Name: lang, dtype: int64